In [20]:
from google.colab import drive
drive.mount('/content/drive')

inputfile = open('/content/drive/MyDrive/algoritmoMapReduce_ErikaBrito_GeorgeGuevara/paginas.txt',"r")
text = inputfile.read()
import requests
from bs4 import BeautifulSoup

def extract_text_from_web_pages(text):
    urls = text.split()
    extracted_text = ""

    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            text = ' '.join([p.get_text() for p in soup.find_all('p')])
            extracted_text += text + "\n"

    return extracted_text

def data_clean(text):
  NoNumbers = ''.join([i for i in text if not i.isdigit()])
  NoNumbers = text.lower()
  import re
  onlyText = re.sub(r"[^a-z\s]+",' ',NoNumbers)
  finaltext = "".join([s for s in onlyText.strip().splitlines(True) if s.strip()])
  return finaltext
def splitlines(text,a):
    linessplit = text.splitlines()
    split1 = linessplit[0:a]
    split2 = linessplit[a:]
    return split1,split2

def mapper(text):
    keyval = []
    for i in text:
      wordssplit = i.split()
      for j in wordssplit:
        keyval.append([j,1])
    return keyval

def sortedlists(list1,list2):
    out = list1 + list2
    out.sort(key= lambda x :x[0])
    return out

def partition(sorted_list) :
    sort1out = []
    sort2out = []
    for i in sorted_list:
        if i[0][0] < 'n':
          sort1out.append(i)
        else : sort2out.append(i)
    return sort1out,sort2out
def reducer(part_out1) :
  sum_reduced = []
  count = 1
  for i in range(0,len(part_out1)):
    if i < len(part_out1)-1:
      if part_out1[i] == part_out1[i+1]:
       count = count+1
      else :
       sum_reduced.append([part_out1[i][0],count])
       count = 1
    else: sum_reduced.append(part_out1[i])
  return sum_reduced
def hilo (text):
    textweb=extract_text_from_web_pages(text)
    cleantext = data_clean(textweb)
    splittext = splitlines(cleantext,5000)
    map1out = mapper(splittext[0])
    map2out = mapper(splittext[1])
    sortedwords = sortedlists(map1out,map2out)
    slicedwords = partition(sortedwords)
    reducerout1 = reducer(slicedwords[0])
    reducerout2 = reducer(slicedwords[1])
    return reducerout1+reducerout2
output = hilo(text)
import pandas as pd

pd.DataFrame(output).to_csv("/content/drive/MyDrive/algoritmoMapReduce_ErikaBrito_GeorgeGuevara/nuevoconteo.csv",index=False,header = ["Word","Frequency"]) #Saving file as a .csv file in the current directory

import threading
def main():
    un_hilo=threading.Thread(target=hilo)
    un_hilo.start()
    un_hilo.join()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
